In [3]:
###############################################################################
# VectorBT strategy + QuantStats tear‑sheet (RELIANCE.NS vs NIFTY‑50)
###############################################################################
import warnings, yfinance as yf, quantstats as qs, vectorbt as vbt
import pandas as pd
import numpy  as np
from pandas.core.resample import Resampler

# ── 1. one‑time pandas‑2.2 axis‑kwarg patch (skip if already in your session) ──
if not getattr(Resampler, "_qs_axis_patch", False):
    _orig_sum = Resampler.sum
    Resampler.sum = lambda self, *a, **k: _orig_sum(self, *a)   # drop axis kwarg
    Resampler._qs_axis_patch = True
# ───────────────────────────────────────────────────────────────────────────────

warnings.filterwarnings(
    "ignore",
    message="A value is trying to be set on a copy of a DataFrame or Series",
    category=FutureWarning,
    module=r"quantstats\._plotting\.core"
)

# ── 2. pull price data (RELIANCE & NIFTY‑50) ──────────────────────────────────
tickers  = ["RELIANCE.NS", "^NSEI"]
start_dt = "2018-01-01"

raw = yf.download(tickers, start=start_dt, progress=False)

def get_close(df, ticker):
    lvl0 = df.columns.get_level_values(0)
    col0 = "Adj Close" if "Adj Close" in lvl0 else "Close"
    return df.xs(col0, level=0, axis=1)[ticker]

rel_close  = get_close(raw, "RELIANCE.NS")
nifty_close = get_close(raw, "^NSEI")

# ── 3. run EMA‑crossover strategy in vectorbt ─────────────────────────────────
short_ema = vbt.MA.run(rel_close, window=2,  ewm=True, short_name='fast')
long_ema  = vbt.MA.run(rel_close, window=27, ewm=True, short_name='slow')

entries = short_ema.ma_crossed_above(long_ema)
exits   = short_ema.ma_crossed_below(long_ema)

initial_capital = 1_000_000

pf = vbt.Portfolio.from_signals(     # <‑‑ fix is right here
    close            = rel_close,    # ① supply price series via `close=…`  (or positional)
    entries          = entries,
    exits            = exits,
    direction        = 'longonly',
    size_type        = 'percent',
    size             = 1.00,         # 100 % of equity on each entry
    fees             = 0.0012,
    init_cash        = initial_capital,
    freq             = '1D',
    min_size         = 1,
    size_granularity = 1
)


# ── 4. build QuantStats tear‑sheet from PF returns ────────────────────────────
# • pf.returns is already daily; drop the first NaN to placate QuantStats
strat_ret  = pf.returns().dropna()
bench_ret  = nifty_close.pct_change().dropna()

# align by inner‑join so both Series have identical dates
strat_ret, bench_ret = strat_ret.align(bench_ret, join="inner")

qs.reports.html(
    returns   = strat_ret,
    benchmark = bench_ret,
    title     = "EMA‑Crossover on RELIANCE.NS vs NIFTY 50",
    output    = "reliance_ema_tearsheet.html"
)

print("\n✅  Tear‑sheet saved to reliance_ema_tearsheet.html")


C:\Users\Admin1\AppData\Local\Temp\ipykernel_86680\3416737710.py:64: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.




✅  Tear‑sheet saved to reliance_ema_tearsheet.html


In [4]:
stats = pf.stats()
print("Backtest Stats:")
print(stats)


Backtest Stats:
Start                               2018-01-01 00:00:00
End                                 2025-05-06 00:00:00
Period                               1811 days 00:00:00
Start Value                                   1000000.0
End Value                                 2324349.33929
Total Return [%]                             132.434934
Benchmark Return [%]                         246.215768
Max Gross Exposure [%]                        99.999436
Total Fees Paid                            312602.96497
Max Drawdown [%]                              34.219724
Max Drawdown Duration                 304 days 00:00:00
Total Trades                                         73
Total Closed Trades                                  72
Total Open Trades                                     1
Open Trade PnL                            294484.453387
Win Rate [%]                                  22.222222
Best Trade [%]                                81.771093
Worst Trade [%]                 